## **Sequence-to-sequence learning : EN to FR translation** - revision 01

### **1. Small_vocab translation**

In [24]:
import os
import time

def load_data(path):

    # Nombre maximum de lignes à renvoyer
    max_lines = 140000
    
    input_file = os.path.join(path)
    with open(input_file, "r",  encoding="utf-8") as f:
        data = f.read()
        
    # On convertit les majuscules en minulcule
    data = data.lower()
    
    data = data.split('\n')
    return data[:min(len(data),max_lines)]

# Indication de calculer le score Bleu
bleu_score_to_do = True

# Chargement des textes dans les 2 langues (max lignes = max_lines)
txt_en = load_data('../data/small_vocab_en')
txt_fr = load_data('../data/small_vocab_fr')

text_pairs = []
for line in range(len(txt_en)):
    txt_fr[line]=txt_fr[line].replace('à', 'a')
    text_pairs.append((txt_en[line], "[start] " + txt_fr[line] + " [end]" ))
    

In [2]:
import random
print(random.choice(text_pairs))

('he wants to go to paris next october .', '[start] il veut se rendre a paris en octobre prochain . [end]')


In [3]:
import random
random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples:num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples:]

**Vectorizing the English and French text pairs**

In [4]:
import tensorflow as tf
import string
import re
from tensorflow import keras
from tensorflow.keras import layers

strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(
        lowercase, f"[{re.escape(strip_chars)}]", "")

def save_vocab(file_path, vocab):
    with open(file_path, "w",  encoding="utf-8" ) as file:
        for word in vocab:
            file.write(word+'\n')
    return

vocab_size = 15000
sequence_length = 30

source_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)
target_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
)
train_english_texts = [pair[0] for pair in train_pairs]
train_french_texts = [pair[1] for pair in train_pairs]
source_vectorization.adapt(train_english_texts)
target_vectorization.adapt(train_french_texts)

save_vocab("data/eng_vocab.txt" , source_vectorization.get_vocabulary()[2:])
save_vocab("data/fra_vocab.txt" , target_vectorization.get_vocabulary()[2:])


**Preparing datasets for the translation task**

In [5]:
batch_size = 64

def format_dataset(eng, fra):
    eng = source_vectorization(eng)
    fra = target_vectorization(fra)
    return ({
        "english": eng,
        "french": fra[:, :-1],
    }, fra[:, 1:])

def make_dataset(pairs):
    eng_texts, fra_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    fra_texts = list(fra_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, fra_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset, num_parallel_calls=4)
    return dataset.shuffle(2048).prefetch(16).cache()

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

In [6]:
for inputs, targets in train_ds.take(1):
    print(f"inputs['english'].shape: {inputs['english'].shape}")
    print(f"inputs['french'].shape: {inputs['french'].shape}")
    print(f"targets.shape: {targets.shape}")

inputs['english'].shape: (64, 30)
inputs['french'].shape: (64, 30)
targets.shape: (64, 30)


### **2. Sequence-to-sequence learning with RNNs**

**GRU-based encoder**

In [7]:
from tensorflow import keras
from tensorflow.keras import layers

embed_dim = 256
latent_dim = 1024

source = keras.Input(shape=(None,), dtype="int64", name="english")
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(source)
encoded_source = layers.Bidirectional(
    layers.GRU(latent_dim), merge_mode="sum")(x)

**GRU-based decoder and the end-to-end model**

In [8]:
past_target = keras.Input(shape=(None,), dtype="int64", name="french")
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(past_target)
decoder_gru = layers.GRU(latent_dim, return_sequences=True)
x = decoder_gru(x, initial_state=encoded_source)
x = layers.Dropout(0.5)(x)
target_next_step = layers.Dense(vocab_size, activation="softmax")(x)
seq2seq_rnn = keras.Model([source, past_target], target_next_step)

**Training our recurrent sequence-to-sequence model**

In [9]:
seq2seq_rnn.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])
stop_early = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
seq2seq_rnn.fit(train_ds, epochs=15, validation_data=val_ds, callbacks=[stop_early])

Epoch 1/15
1508/1508 [==============================] - 224s 130ms/step - loss: 0.2896 - accuracy: 0.8232 - val_loss: 0.0367 - val_accuracy: 0.9762
Epoch 2/15
1508/1508 [==============================] - 184s 122ms/step - loss: 0.0316 - accuracy: 0.9811 - val_loss: 0.0192 - val_accuracy: 0.9882
Epoch 3/15
1508/1508 [==============================] - 185s 123ms/step - loss: 0.0208 - accuracy: 0.9878 - val_loss: 0.0145 - val_accuracy: 0.9912
Epoch 4/15
1508/1508 [==============================] - 184s 122ms/step - loss: 0.0162 - accuracy: 0.9904 - val_loss: 0.0128 - val_accuracy: 0.9925
Epoch 5/15
1508/1508 [==============================] - 184s 122ms/step - loss: 0.0137 - accuracy: 0.9919 - val_loss: 0.0111 - val_accuracy: 0.9935
Epoch 6/15
1508/1508 [==============================] - 183s 121ms/step - loss: 0.0120 - accuracy: 0.9929 - val_loss: 0.0103 - val_accuracy: 0.9941
Epoch 7/15
1508/1508 [==============================] - 186s 123ms/step - loss: 0.0108 - accuracy: 0.9936 - val_

**Saving of the trained RNN model**

In [10]:
seq2seq_rnn.save("../data/seq2seq_rnn-fra-en-model.h5")

**Translating new sentences with our RNN encoder and decoder**

In [21]:
import numpy as np
fra_vocab = target_vectorization.get_vocabulary()
fra_index_lookup = dict(zip(range(len(fra_vocab)), fra_vocab))
max_decoded_sentence_length = 30

def decode_sequence_RNN(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization([decoded_sentence])
        next_token_predictions = seq2seq_rnn.predict(
            [tokenized_input_sentence, tokenized_target_sentence], verbose=0)
        sampled_token_index = np.argmax(next_token_predictions[0, i, :])
        sampled_token = fra_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        if sampled_token == "[end]":
            break
    return decoded_sentence[8:-6]

test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(20):
    input_sentence = random.choice(test_eng_texts)
    print("-")
    print("EN  ",input_sentence)
    print("FR->",decode_sequence_RNN(input_sentence))

-
EN   france is usually cold during july , but it is never mild in autumn .
FR-> la france est généralement froid en juillet mais il est jamais doux a l automne
-
EN   india is usually dry during january , and it is never snowy in september .
FR-> l inde est généralement sec en janvier et il est jamais neigeux en septembre
-
EN   california is sometimes wonderful during july , and it is relaxing in june .
FR-> californie est parfois merveilleux en juillet et il est relaxant en juin
-
EN   the pear is her most loved fruit , but the grape is his most loved .
FR-> la poire est son fruit le plus aimé mais le raisin est le plus aimé
-
EN   france is chilly during november , and it is snowy in fall .
FR-> la france est froid en novembre et il est neigeux a l automne
-
EN   new jersey is cold during autumn , but it is freezing in may .
FR-> new jersey est froid pendant l automne mais il gèle en mai
-
EN   france is never rainy during march , and it is hot in december .
FR-> la france est jam

### **3. Sequence-to-sequence learning with Transformer**

#### The Transformer decoder

**The `TransformerDecoder`**

In [12]:
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1),
             tf.constant([1, 1], dtype=tf.int32)], axis=0)
        return tf.tile(mask, mult)

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(
                mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)
        else:
            padding_mask = mask
        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=causal_mask)
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        attention_output_2 = self.layernorm_2(
            attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)

#### Putting it all together: A Transformer for machine translation

**PositionalEmbedding layer**

In [13]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super(PositionalEmbedding, self).get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

**End-to-end Transformer**

In [14]:
from keras_nlp.layers import TransformerEncoder
embed_dim = 256
dense_dim = 2048
num_heads = 8

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="english")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim, name='positionalEmbeddingEnglish')(encoder_inputs)
encoder_outputs = TransformerEncoder(dense_dim, num_heads, name='encoder_outputs')(x)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="french")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim, name='positionalEmbeddingFrench')(decoder_inputs)
x = TransformerDecoder(embed_dim, dense_dim, num_heads,  name='transformerDecoder')(x, encoder_outputs)
x = layers.Dropout(0.5, name='dropout')(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax", name='decoder_outputs')(x)
transformer = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

Using TensorFlow backend


**Training the sequence-to-sequence Transformer**

In [15]:
transformer.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])
stop_early = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
transformer.fit(train_ds, epochs=15, validation_data=val_ds, callbacks=[stop_early])

Epoch 1/15
1508/1508 [==============================] - 157s 98ms/step - loss: 0.1585 - accuracy: 0.9220 - val_loss: 0.0370 - val_accuracy: 0.9752
Epoch 2/15
1508/1508 [==============================] - 145s 96ms/step - loss: 0.0382 - accuracy: 0.9771 - val_loss: 0.0292 - val_accuracy: 0.9816
Epoch 3/15
1508/1508 [==============================] - 135s 90ms/step - loss: 0.0307 - accuracy: 0.9819 - val_loss: 0.0290 - val_accuracy: 0.9819
Epoch 4/15
1508/1508 [==============================] - 140s 93ms/step - loss: 0.0263 - accuracy: 0.9846 - val_loss: 0.0196 - val_accuracy: 0.9878
Epoch 5/15
1508/1508 [==============================] - 145s 96ms/step - loss: 0.0228 - accuracy: 0.9866 - val_loss: 0.0218 - val_accuracy: 0.9866
Epoch 6/15
1508/1508 [==============================] - 145s 96ms/step - loss: 0.0202 - accuracy: 0.9881 - val_loss: 0.0198 - val_accuracy: 0.9882
Epoch 7/15
1508/1508 [==============================] - 145s 96ms/step - loss: 0.0178 - accuracy: 0.9896 - val_loss: 0

**Saving of the trained Transformer model**

In [16]:
for i in range(len(transformer.weights)):
    transformer.weights[i]._handle_name = transformer.weights[i].name + "_" + str(i)
    
transformer.save("../data/transformer-fra-en-model.h5", include_optimizer=False)
transformer.save_weights("../data/transformer-fra-en-model.weights.h5")


**Translating new sentences with our Transformer model**

In [52]:
import numpy as np
fra_vocab = target_vectorization.get_vocabulary()
fra_index_lookup = dict(zip(range(len(fra_vocab)), fra_vocab))
max_decoded_sentence_length = 30

def decode_sequence_Transformer(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization(
            [decoded_sentence])[:, :-1]
        predictions = transformer(
            [tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = fra_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        if sampled_token == "[end]":
            break
    return decoded_sentence[8:-6]

test_eng_texts = [pair[0] for pair in test_pairs]
test_fra_texts = [pair[1][8:-6] for pair in test_pairs]

# Removing Punctuations (we keep the . character)
REPLACEMENTS = [(".", ""),
                (",", ""),
                (";", ""),
                (":", ""),
                ("?", ""),
                ('"', ""),
                ("-", " "),
                ("'", " ")
               ]
for i in range(len(test_fra_texts)):
    for old, new in REPLACEMENTS:
        test_fra_texts[i] = test_fra_texts[i].replace(old, new)

for _ in range(20):
    n = random.randrange(num_val_samples)
    input_sentence = test_eng_texts[n]
    translation = test_fra_texts[n]
    print("-")
    print("EN  ",input_sentence)
    print("FR  ",translation)
    print("FR->",decode_sequence_Transformer(input_sentence))

-
EN   he likes apples and grapes .
FR   il aime les pommes et les raisins 
FR-> il aime les pommes et les raisins
-
EN   new jersey is never quiet during july , and it is sometimes wonderful in autumn .
FR   new jersey est jamais tranquille en juillet  et il est parfois merveilleux en automne 
FR-> new jersey est jamais tranquille en juillet et il est parfois merveilleux en automne
-
EN   california is sometimes dry during autumn , and it is sometimes nice in april .
FR   californie est parfois sec pendant l  automne  et il est parfois agréable en avril 
FR-> californie est parfois sec pendant l automne et il est parfois agréable en avril
-
EN   india is sometimes rainy during june , but it is warm in spring .
FR   l  inde est parfois pluvieux en juin  mais il est chaud au printemps 
FR-> l inde est parfois pluvieux en juin mais il est chaud au printemps
-
EN   she drives the rusty white truck .
FR   elle conduit le camion blanc rouillé 
FR-> elle conduit le camion blanc rouillé
-
EN 

<br></br>
**You turn to play:** Enter a sentence..

In [53]:
entence = "india is never cold during winter , but it is beautiful in autumn"
print("FR->",decode_sequence_Transformer(input_sentence))

FR-> elle est au volant d un nouveau camion blanc


**Qualité de traduction seq2seq sur le corpus Test par les 2 méthodes (300 premieres phrases)**

In [54]:
from sacrebleu import sentence_bleu, corpus_bleu

# Taille du corpus d'évaluation
n = 300

def corpus_translation(method):
    translation = []
    test_eng_texts = [pair[0] for pair in test_pairs]
    for txt in test_eng_texts[:n]:
        if method == 'RNN':
            translation.append(decode_sequence_RNN(txt))
        else:
            translation.append(decode_sequence_Transformer(txt))
    return translation

if bleu_score_to_do:
    tps1 = time.time()
    tr_RNN = corpus_translation('RNN')
    translation_quality_RNN = corpus_bleu(tr_RNN, [test_fra_texts[:n]]).score
    print(f'Score Bleu de la traduction du corpus Test par la méthode RNN = {translation_quality_RNN:.1f}%')
    tps2 = time.time()
    print(" -> durée de traduction et de calcul du score :",int(tps2-tps1)," secondes\n")
    tr_Transformer = corpus_translation('Transformer')
    translation_quality_Transformer = corpus_bleu(tr_Transformer, [test_fra_texts[:n]]).score
    print(f'Score Bleu de la traduction du corpus Test par la méthode Transformer = {translation_quality_Transformer:.1f}%')
    tps3 = time.time()
    print(" -> durée de traduction et de calcul du score :",int(tps3-tps2)," secondes\n")


Score Bleu de la traduction du corpus Test par la méthode RNN = 94.5%
 -> durée de traduction et de calcul du score : 1064  secondes

Score Bleu de la traduction du corpus Test par la méthode Transformer = 93.4%
 -> durée de traduction et de calcul du score : 558  secondes

